In [ ]:
import requests, pickle
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras import Input
import tensorflow.keras.optimizers as optimizers
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot
from keras.src.layers import Dropout
from keras.src.regularizers import regularizers

### Carga de datos

In [ ]:
url = 'https://github.com/greydanus/mnist1d/raw/master/mnist1d_data.pkl'
r = requests.get(url, allow_redirects=True)
open('./mnist1d_data.pkl', 'wb').write(r.content)

with open('./mnist1d_data.pkl', 'rb') as handle:
    data = pickle.load(handle)
    
data.keys()

### Desempaquetado y normalización de los datos
MinMaxScaler de sklearn y OneHot encoder de autoría propia

In [ ]:
import mios
x = data["x"]
x_test = data["x_test"]
y = data["y"]
y_test = data["y_test"]
y = mios.OneHot(y)
y_test = mios.OneHot(y_test)

scaler = MinMaxScaler()
scaler.fit(x)
scaler.transform(x)
scaler.transform(x_test)
print()

### Declaración de la red neuronal

In [ ]:
PROP = Sequential()
PROP.add(Input(shape=(40,)))
PROP.add(Dropout(0.2, input_dim=40))
PROP.add(Dense(40, activation='relu', activity_regularizer=regularizers.L2(0.00001)))
PROP.add(Dense(40, activation='relu', activity_regularizer=regularizers.L2(0.00001)))
PROP.add(Dense(40, activation='relu', activity_regularizer=regularizers.L2(0.00001)))
PROP.add(Dense(10, activation='softmax'))

callbacks = [keras.callbacks.EarlyStopping(patience=200)]

# opti = optimizers.Adam(0.001)
opti = optimizers.SGD(learning_rate=0.05, momentum=0.01)

PROP.compile(loss=keras.losses.CategoricalCrossentropy(),
              optimizer=opti,
              metrics=['accuracy'])

# PROP.summary()

H = PROP.fit(x, y, epochs=1000, validation_data=[x_test, y_test], callbacks=callbacks)

### Graficación de comportamiento

In [ ]:
pyplot.title("Exactitud")
pyplot.plot(H.history['accuracy'], label='snn_tr')
pyplot.plot(H.history['val_accuracy'], label='snn_va')
pyplot.ylim(-0.1,1.1)
pyplot.legend()
pyplot.show()

pyplot.title("Pérdida")
pyplot.plot(H.history['loss'], label='snn_tr')
pyplot.plot(H.history['val_loss'], label='snn_va')
pyplot.ylim(-0.1,10.1)
pyplot.legend()
pyplot.show()

In [ ]:
PROP.summary()